In [20]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [21]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [22]:
tf.__version__

'2.4.1'

In [23]:
fashion_mnist = keras.datasets.fashion_mnist

In [24]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [25]:
train_images = train_images/255.0
test_images = test_images/255.0

In [38]:
train_images[0].shape

(28, 28)

In [39]:
train_images = train_images.reshape(len(train_images), 28,28,1)
test_images = test_images.reshape(len(test_images), 28,28,1)

In [44]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value =32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value =32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu',
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value =32, max_value=128, step=16),
        activation='relu',
    ),
    keras.layers.Dense(10, 'softmax')
  ])

  model.compile(
      optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
      loss = keras.losses.sparse_categorical_crossentropy
      # loss='sparse_categorical_crossentropy', 
      metrics=['accuracy']
  )

  return model

In [45]:
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [46]:
tuner = RandomSearch(
    build_model, 
    objective='val_accuracy', 
    max_trials=5, 
    directory='output', 
    project_name='Mnist Fashion'
    )

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json


In [47]:
tuner.search(train_images, train_labels, epochs=3, validation_split=0.1)

Trial 5 Complete [00h 00m 21s]
val_accuracy: 0.9104999899864197

Best val_accuracy So Far: 0.9141666889190674
Total elapsed time: 00h 02m 21s
INFO:tensorflow:Oracle triggered exit


In [51]:
model=tuner.get_best_models(num_models=1)[0]

In [52]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 48)        480       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        57648     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1486912   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 1,545,690
Trainable params: 1,545,690
Non-trainable params: 0
_________________________________________________________________


In [53]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1397 - accuracy: 0.9475 - val_loss: 0.2735 - val_accuracy: 0.9138
Epoch 5/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1003 - accuracy: 0.9624 - val_loss: 0.2868 - val_accuracy: 0.9135
Epoch 6/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0718 - accuracy: 0.9741 - val_loss: 0.3321 - val_accuracy: 0.9138
Epoch 7/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0536 - accuracy: 0.9804 - val_loss: 0.3674 - val_accuracy: 0.9138
Epoch 8/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0414 - accuracy: 0.9848 - val_loss: 0.4266 - val_accuracy: 0.9142
Epoch 9/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0321 - accuracy: 0.9884 - val_loss: 0.4813 - val_accuracy: 0.9143
Epoch 10/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0229 - accuracy: 0.9918 - val_loss: 0.5189 - val_accuracy

In [54]:
model

In [55]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 48)        480       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        57648     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1486912   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 1,545,690
Trainable params: 1,545,690
Non-trainable params: 0
_________________________________________________________________
